# Aleasim Simple Control Test

In [ ]:
%load_ext autoreload
%autoreload 2

from alea.sim.epa.disturbance_model import DisturbanceModel
from alea.sim.math_lib.math_ahrs import AHRSQuaternion

import logging

import alea.sim
import numpy as np
import scipy as sp
import skyfield

from datetime import datetime
import time
from alea.sim.epa.magnetic_field_model import EarthMagneticFieldModel
from alea.sim.epa.attitude_dynamics import AttitudeDynamicsModel
from alea.sim.epa.orbit_dynamics import OrbitDynamicsModel
from alea.sim.kernel.kernel import AleasimKernel
from alea.sim.spacecraft.spacecraft import Spacecraft
import matplotlib.pyplot as plt
import logging
import cProfile
import pstats
from pstats import SortKey
from matplotlib import pyplot as plt
import skyfield.sgp4lib

from typing import Dict

import numpy as np


from alea.sim.kernel.frames import ReferenceFrame, FrameTransformation
from alea.sim.spacecraft.actuators.simple_actuators import SimpleActuator, SimpleMagnetorquer
from alea.sim.spacecraft.sensors.simple_sensors import SimpleMagSensor, SimpleSunSensor, SimpleGyroSensor

logging.basicConfig(level=logging.INFO)

In [ ]:
%load_ext autoreload
%autoreload 2

sim_dt=1e-2 #s
control_dt=0.1 #s
duration=10.0 #s
st=0.284e-3 #Nm
sm=0 #Am^2


control_sample_rate = int(1/control_dt)
kernel = AleasimKernel(dt=sim_dt, date=2024.0)

adyn = AttitudeDynamicsModel(kernel)
odyn = OrbitDynamicsModel(kernel)

magm = EarthMagneticFieldModel(kernel)

rwx = SimpleActuator('rw_x', kernel, st)
rwy = SimpleActuator('rw_y', kernel, st)
rwz = SimpleActuator('rw_z', kernel, st)

mtqx = SimpleMagnetorquer('mtq_x',kernel, sm)
mtqy = SimpleMagnetorquer('mtq_y',kernel, sm)
mtqz = SimpleMagnetorquer('mtq_z',kernel, sm)

sc = Spacecraft(kernel, ctrl_sample_period=control_dt)

DModel = DisturbanceModel(kernel)

mag_sens = SimpleMagSensor('mag_sens', kernel, sample_rate=control_sample_rate)
sun_sens = SimpleSunSensor('sun_sens', kernel, sample_rate=control_sample_rate)
gyro = SimpleGyroSensor('gyro', kernel, sample_rate=control_sample_rate)
kernel.add_model(rwx)
kernel.add_model(rwy)
kernel.add_model(rwz)
kernel.add_model(mtqx)
kernel.add_model(mtqy)
kernel.add_model(mtqz)
kernel.add_model(sc, True)
kernel.add_model(magm)
kernel.add_model(odyn, True)
kernel.add_model(adyn, True)
kernel.add_model(DModel)
kernel.add_model(mag_sens)
kernel.add_model(sun_sens)
kernel.add_model(gyro)
adyn._integrator_type = "rk45"
adyn.set_state(np.array([0.1,0.756,0.1,0,0,0,0.0]))
kernel.set_log_level_all(logging.INFO)
kernel.step()
sc._use_quest = True

kernel.advance(duration)

kernel.kill()

In [ ]:
print(f'mean error {np.mean(sc.state_array[:,10])}')
objs = plt.plot(sc.time_array, sc.state_array[:,10], label=sc.saved_state_element_names[10])
plt.title("Attitude Quaternion Estimate Error")
plt.xlabel('Time (s)')
plt.ylabel('degrees')
plt.show()

fig, axs = plt.subplots(4)
for i in range(4):
    ax = axs[i]
    ax.plot(sc.time_array, sc.state_array[:,6+i], label=f'q{i}_estimated', linestyle='dashed')
    ax.plot(adyn.time_array, adyn.state_array[:,i], label=f'q{i}_true',linewidth=3)
    ax.legend()
plt.suptitle("Estimated and Real Quaternion Elements")
plt.xlabel('Time (s)')
plt.show()

objs = plt.plot(adyn.time_array, adyn.state_array[:,4:7])
plt.title("Angular Rates")
plt.legend(iter(objs), ('w1', 'w2', 'w3'))
plt.ylabel('rad/s')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,7:10])
plt.title("Angular Acceleration")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[7:10]))
plt.ylabel('rad/s^2')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(adyn.time_array, adyn.state_array[:,10:])
plt.title("Disturbance Torques")
plt.legend(iter(objs), tuple(adyn.saved_state_element_names[10:13]))
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(sc.time_array, sc.state_array[:,0:3])
plt.title("Reaction Wheel Torques")
plt.legend(iter(objs), tuple(sc.saved_state_element_names[0:3]))
plt.ylabel('Nm')
plt.xlabel('Time (s)')
plt.show()


objs = plt.plot(sc.time_array, sc.state_array[:,3:6])
plt.title("Mtq Dipole Moments")
plt.legend(iter(objs), tuple(sc.saved_state_element_names[3:6]))
plt.ylabel('Am^2')
plt.xlabel('Time (s)')
plt.show()

sensors = [gyro, sun_sens, mag_sens]
for sens in sensors:
    fig, (ax1, ax2) = plt.subplots(2, sharex=True)
    for i in range(3):
        ax1.plot(sens.time_array, sens.state_array[:,i], label=sens.saved_state_element_names[i])
        ax2.plot(sens.time_array, sens.state_array[:,i+sens.axes], label=sens.saved_state_element_names[i+sens.axes])
        ax1.legend()
        ax2.legend()
    plt.suptitle(f'{sens.name} Measurements vs. Ground Truth')
    plt.xlabel('Time (s)')
    plt.show()